# In this notebook the function that calculates L was tested

In [1]:
import numpy as np
import os
import pickle
from numba import jit
import math
import cmath

import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [2]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
def save_psi_array(array, filename):
    pickle.dump(array, open("saved_simulations/"+filename, "wb"))

In [4]:
def display_psi_array(array, dynamic_colorbar=True):
    fig = plt.figure()
    shape = array[0].shape
    data = np.random.rand(shape[0],shape[1])
    vmin = np.min(array[0])
    vmax = np.max(array[0])
    im = plt.imshow(data, cmap="jet", animated=True, vmin=vmin, vmax=vmax)
    plt.colorbar()
    def update(i):
        im.set_data(array[i])
        if dynamic_colorbar:
            vmin = np.min(array[i])
            vmax = np.max(array[i])
            im.set_clim(vmin, vmax)
        return im
    anim = animation.FuncAnimation(fig, update, frames=len(array), interval=50)
    plt.show()
    return anim

In [5]:
def gauss(x, y, x0=0, y0=0):
    return 1/np.sqrt(np.pi) * np.exp(-0.5*((x-x0)**2+(y-y0)**2))

In [65]:
N = 101
a = -10
b = 10
x = np.linspace(a,b, N)
y = x

In [92]:
xx, yy = np.meshgrid(x, y)
psi = gauss(xx, yy, 2, 4) + (0+0j)
plt.imshow(np.abs(psi))
plt.colorbar()
plt.show()

Step by step trying to understand how to speed up the calculation of L with numpy

In [102]:
5/0.005

1000.0

In [86]:
@jit
def L(psi, psi_hat, a, N):
    psi_dx = np.ones((N,N)) + (0+0j)
    psi_dy = np.ones((N,N)) + (0+0j)
    psi_L = np.ones((N,N)) + (0+0j)
    
    for j in range(N):
        for k in range(N):
            s_dx = 0
            s_dy = 0
            for p in range(-N//2, N//2):
                for q in range(-N//2, N//2):
                    my_p = 2*p*math.pi/(2*a)
                    lambda_q = 2*q*math.pi/(2*a)
                    tmp = psi_hat[p, q] * cmath.exp(1j*2*j*p*np.pi/N) * cmath.exp(1j*2*k*q*np.pi/N)
                    s_dx += my_p * tmp
                    s_dy += lambda_q * tmp
            #print('error?')
            #print(my_p.shape)
            #print(s_dx.shape)
            psi_dx[j,k] = s_dx / (N**2)
            psi_dy[j,k] = s_dy / (N**2)
            #print("Error?")
    #bad! ... ?
    dx = 2*a/N
    x = np.arange(-a, a, dx)
    y = np.arange(-a, a, dx)
    xx, yy = np.meshgrid(x, y, sparse=False, indexing='xy')

    psi_L = xx * psi_dy - yy * psi_dx
    return psi_L


In [87]:
psi_hat = np.fft.fft2(psi)#/N**2
l = L(psi, psi_hat, a, N)
# plt.imshow(np.abs(l)**2)
plt.imshow(np.abs(l))
plt.colorbar()
plt.show()

In [93]:
@jit
def L_semi(psi, psi_hat, a, N):
    psi_dx = np.ones((N,N)) + (0+0j)
    psi_dy = np.ones((N,N)) + (0+0j)
    psi_dx_hat = np.ones((N,N)) + (0+0j)
    psi_dy_hat = np.ones((N,N)) + (0+0j)
    psi_L = np.ones((N,N)) + (0+0j)
    
    for p in range(-N//2, N//2):
        for q in range(-N//2, N//2): 
            my_p = 2*p*math.pi/(2*a)
            lambda_q = 2*q*math.pi/(2*a)
            psi_dx_hat[p,q] = psi_hat[p, q] * my_p
            psi_dy_hat[p,q] = psi_hat[p, q] * lambda_q
    
    psi_dx = np.fft.ifft2(psi_dx_hat)# * N**2
    psi_dy = np.fft.ifft2(psi_dy_hat) #* N**2
    
    #psi_dx = np.fft.fftshift(psi_dx)
    #psi_dy = np.fft.fftshift(psi_dy)
    
    # combine the two
    x = np.linspace(-a, a, N)
    y = np.linspace(-a, a, N)
    xx, yy = np.meshgrid(x, y, sparse=False, indexing='xy')

    psi_L = xx * psi_dy - yy * psi_dx
    return psi_L

In [94]:
psi_hat = np.fft.fft2(psi)#/N**2
#psi_hat = psi_hat.real

l = L_semi(psi, psi_hat, a, N)
plt.imshow(np.abs(l))
plt.colorbar()
plt.show()

In [95]:
@jit
def L_fast(psi, psi_hat, a, N):
    psi_dx = np.ones((N,N)) + (0+0j)
    psi_dy = np.ones((N,N)) + (0+0j)
    psi_dx_hat = np.ones((N,N)) + (0+0j)
    psi_dy_hat = np.ones((N,N)) + (0+0j)
    psi_L = np.ones((N,N)) + (0+0j)
    
    p = np.arange(-N//2, N//2)
    q = np.arange(-N//2, N//2)
    
    pp, qq = np.meshgrid(p, q, sparse=False, indexing="ij")
    
    my_p = 2*pp*math.pi/(2*a)
    lambda_q = 2*qq*math.pi/(2*a)
    
    my_p = np.fft.fftshift(my_p)
    lambda_q = np.fft.fftshift(lambda_q)
    
    psi_dx_hat = psi_hat * my_p
    psi_dy_hat = psi_hat * lambda_q
    
    
    psi_dx = np.fft.ifft2(psi_dx_hat)# * N**2
    psi_dy = np.fft.ifft2(psi_dy_hat) #* N**2
    

    dx = 2*a/N
    x = np.arange(-a, a, dx)
    y = np.arange(-a, a, dx)
    xx, yy = np.meshgrid(x, y, sparse=False, indexing='xy')

    psi_L = xx * psi_dy - yy * psi_dx
    return psi_L

In [96]:
psi_hat = np.fft.fft2(psi)#/N**2
l = L_fast(psi, psi_hat, a, N)
# plt.imshow(np.abs(l)**2)
plt.imshow(np.abs(l))
plt.colorbar()
plt.show()